In [1]:
import requests, os, imagehash, time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from PIL import Image, ImageChops

In [2]:
webdrive_path = 'webdrive\chromedriver.exe'
origin_folder_path = 'tiger_origin'
image_folder_path = 'tiger'

new_size = 256
crop_f = 1.5

In [ ]:
def get_images_urls(url):
    driver = webdriver.Chrome(executable_path = webdrive_path)
    driver.get(url)
    driver.implicitly_wait(30)

    for _ in range(15):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)

    images = driver.find_elements_by_css_selector('div.imgpt img')
    
    images_link_list = []

    for image in images:
        images_link_list.append(image.get_attribute("src"))
        
    driver.close()
    
    print('Found: ', len(images_link_list))
    
    return images_link_list

# Find image links:

In [ ]:
# cat url at Bing
url_1 = 'https://www.bing.com/images/search?q=cat%20forest&qs=n&form=QBIRMH&qft=%20filterui%3Alicense-L2_L3_L4_L5_L6_L7&sp=-1&pq=cat%20forest&sc=8-10&cvid=5B6ACB06341C4F8B967412A7F4105EE8&first=1&tsc=ImageBasicHover'
url_2 = 'https://www.bing.com/images/search?q=cat%20garden&qs=n&form=QBIRMH&qft=%20filterui%3Alicense-L2_L3_L4_L5_L6_L7&sp=-1&pq=cat%20garden&sc=8-10&cvid=8E62AFD92FD54F0C8456855B60D30EC4&first=1&tsc=ImageBasicHover'
url_3 = 'https://www.bing.com/images/search?q=cat%20home&qs=n&form=QBIRMH&qft=%20filterui%3Alicense-L2_L3_L4_L5_L6_L7&sp=-1&pq=cat%20home&sc=8-8&cvid=10DCAEC5DA7040D68F9A888C165B3A5D&first=1&tsc=ImageBasicHover'

In [ ]:
# tiger url at Bing
url_1 = 'https://www.bing.com/images/search?q=tiger&qft=+filterui:licenseType-Any&form=IRFLTR&first=1&tsc=ImageBasicHover'
url_2 = 'https://www.bing.com/images/search?q=tiger%20wild&qs=n&form=QBIR&qft=%20filterui%3AlicenseType-Any&sp=-1&pq=tiger%20wild&sc=8-10&cvid=1B7286FEE2DA48B98EE9B4238F53EF42&first=1&tsc=ImageBasicHover'
url_3 = 'https://www.bing.com/images/search?q=wild%20tiger%20russia&qs=n&form=QBIRMH&qft=%20filterui%3AlicenseType-Any&sp=-1&pq=wild%20tiger%20russia&sc=0-17&cvid=CB89D5C58817481288A6FFDD342BE29E&first=1&tsc=ImageBasicHover'

In [ ]:
df = pd.DataFrame([])
urls = [url_1, url_2, url_3]

for url in urls:
    list_url = get_images_urls(url)
    df = df.append(list_url, ignore_index=True)

# list_url = get_images_urls(url_1)
# df = df.append(list_url, ignore_index=True)

df = df.drop_duplicates()
df = df.dropna()
df = df[df[0].str.contains("data:")==False]
df.reset_index(drop=True, inplace=True)
    
print('Total found links:', df.shape[0])


In [ ]:
df.to_csv('link.csv', index=False)

In [88]:
df.head()

,0
0,https://th.bing.com/th/id/OIP.m3qQqf9xNgIWxmMk...
1,https://th.bing.com/th/id/OIP.tD5cJfASCM699x9t...
2,https://th.bing.com/th/id/OIP.LBM3Yp6mlyO5o7LT...
3,https://th.bing.com/th/id/OIP.5GerT3NDMrr2ayew...
4,https://th.bing.com/th/id/OIP.QAGUQEqt4ADVCVRf...


# Get images

In [89]:
df = pd.read_csv('link.csv') 
df = pd.DataFrame(df)
print('Total image links:', df.shape[0])

Total image links: 1474


In [56]:
df.loc[2][

'https://th.bing.com/th/id/OIP.LBM3Yp6mlyO5o7LT6GjHSgHaE8?w=238&h=180&c=7&r=0&o=5&pid=1.7'

In [91]:
df[251:500]

,0
251,https://th.bing.com/th/id/OIP.VdRisYeSAwmWgPhE...
252,https://th.bing.com/th/id/OIP.7Rt5YT5R8V_NQoxn...
253,https://th.bing.com/th/id/OIP.mq7vC2hP5THFR8Vg...
254,https://th.bing.com/th/id/OIP.6xXuF4ANrmvdXE3h...
255,https://th.bing.com/th/id/OIP.rSVgR6-eTamY5YD0...
...,...
495,https://th.bing.com/th/id/OIP.IfBTKKbXTvFGgFHX...
496,https://th.bing.com/th/id/OIP.TwAFpi8hfMulkRmo...
497,https://th.bing.com/th/id/OIP.7NZXB0JgU_n4UtkB...
498,https://th.bing.com/th/id/OIP.NE7NlpA0d4uIuYF6...


In [80]:
p = requests.get(df.loc[100][0])

out = open(f"{origin_folder_path}/{100}.jpg", "wb")
out.write(p.content)
out.close()

In [92]:
def get_photo(df, folder_name):
    
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print("Directory " , folder_name ,  " Created ")
        
    i = 1
    for i in range(len(df)):
        p = requests.get(df.loc[i][0])
        out = open(f"{folder_name}/{i}.jpg", "wb")
        out.write(p.content)
        out.close()
    print('Done')

In [93]:
get_photo(df, origin_folder_path)

Done


In [95]:
im=Image.open(f'{origin_folder_path}\\1.jpg') 
im.show()

In [ ]:
# def difference_images(img1, img2):
#     image_1 = Image.open(img1)
#     image_2 = Image.open(img2)
#     result=ImageChops.difference(image_1, image_2).getbbox()
#     if result==None:
#         print(img1,img2,'matches')
#     return

# Find dublicates:

In [98]:
i = 0

hashes = {}
duplicate = {}

for filename in os.listdir(origin_folder_path):
    image = Image.open(f'{origin_folder_path}\\{filename}')
    temp_hash = imagehash.average_hash(image)
    if temp_hash in hashes:
        duplicate[filename] = image
    else:
        hashes[temp_hash] = image
            
print('Found dublicates:', len(duplicate))
# duplicate['1046.jpg'].show()

Found dublicates: 60


In [100]:
# del dublicates
for filename in duplicate.keys():
    os.remove(f'{origin_folder_path}\\{filename}')

# Reshape foto:

In [103]:
def crop_center(pil_img, new_size):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - new_size) // 2,
                         (img_height - new_size) // 2,
                         (img_width + new_size) // 2,
                         (img_height + new_size) // 2))

In [106]:
k = 0

if not os.path.exists(image_folder_path):
        os.makedirs(image_folder_path)
        print("Directory " , image_folder_path ,  " Created ")

for filename in os.listdir(origin_folder_path):
    
    img = Image.open(f'{origin_folder_path}\\{filename}')
    img_w, img_h = img.size
    
    if(img_w < new_size) or (img_h < new_size):
        if img_h > img_w:
            scale = new_size / img_w
            img_w_new = new_size
            img_h_new = int((float(img_h)*float(scale)))
        else:
            scale = new_size / img_h
            img_w_new = int((float(img_w)*float(scale)))
            img_h_new = new_size 
            
        img_new = img.resize((img_w_new, img_h_new), Image.ANTIALIAS)
        
        if img_w_new / new_size >= crop_f or img_h_new / new_size >= crop_f:
            # .crop((left, top, right, bottom))
            
            img_new2 = crop_center(img_new, new_size)
            img_new2.save(f'{image_folder_path}\\{k}.jpg')
            k += 1
            
            img_new2 = img_new.crop((0, 0, new_size, new_size))
            img_new2.save(f'{image_folder_path}\\{k}.jpg')
            k += 1
            
            img_new = img_new.crop((img_w_new - new_size, img_h_new - new_size, img_w_new, img_h_new))
            img_new.save(f'{image_folder_path}\\{k}.jpg')
            k += 1
                        
        else:
            img_new = crop_center(img_new, new_size)
            img_new.save(f'{image_folder_path}\\{k}.jpg')
            k += 1
print('Prepared images:', k)

Directory  tiger  Created 
Prepared images: 1124
